# Config

In [ ]:
path='/media/nvme3n1/proj_scisen/'

In [ ]:
from datasets import Dataset, load_dataset, load_from_disk
import numpy as np

# Load Files

use datasets to load custom dataset

In [ ]:
datasetSci = load_dataset("json",data_files=f'{path}datasets/SciSections_sentences.jsonl')

def filter_as_a(x):
    if x['rank']=='as':
        return True
    if x['rank']=='a':
        return True
    else:
        return False
    
datasetSci = datasetSci.filter(function=filter_as_a)
print('unique sentences: ',sum([len(x) for x in np.unique(datasetSci['train']['sentences'])]))
my_dict = {"text": np.unique([l for x in datasetSci['train']['sentences'] for l in x])}
datasetSci = Dataset.from_dict(my_dict)
datasetSci

# Overwrite existing dataset

In [ ]:
#work around because already open dataset can't be written
def save_dataset(dataset,name):
    from datasets import load_from_disk
    dataset.save_to_disk(f'{path}datasets/style/paraphrases_temp')
    dataset = load_from_disk(f'{path}datasets/style/paraphrases_temp')
    dataset.save_to_disk(f'{path}datasets/style/{name}')
    dataset = load_from_disk(f'{path}datasets/style/{name}')
    #%rm -r f'{path}datasets/style/paraphrases_temp #NOT working 

# Create huggingface dataset

concatenate all the small parts into one big dataset
and split it according to train validation and test

In [ ]:
from datasets import concatenate_datasets
from datasets import DatasetDict
import math

dataset = concatenate_datasets([datasetSci])
#split data and add it to a dictionary
#set fraction to 1 to use the entire dataset otherwise e.g. 0.1
fraction= 1
dataset = DatasetDict({'train':dataset.shuffle(seed=42).select(range(math.floor(len(dataset)*0.7*fraction))),
                       'val':dataset.shuffle(seed=42).select(range(math.floor(len(dataset)*0.7*fraction+1),math.floor(len(dataset)*0.9*fraction))),
                       'test':dataset.shuffle(seed=42).select(range(math.floor(len(dataset)*0.9*fraction+1),math.floor(len(dataset)*fraction)))
                      })
print(dataset)

In [ ]:
#sample output
dataset['train']['text'][:1]
save_dataset(dataset,'dataset')

# Create parallel dataset

In [ ]:
#load dataset from checkpoint - after split with modifications etc.
from datasets import load_from_disk

dataset = load_from_disk(f'{path}datasets/style/dataset')
print(dataset)

## with Pegasus Paraphraser

load model and create method for sample outputs

In [ ]:
import torch
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
model_name = 'tuner007/pegasus_paraphrase'
torch_device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name)
model.to(torch_device)

#create sample outputs for single sentences
def get_response(input_text,num_return_sequences,num_beams):
    batch = tokenizer([input_text],truncation=True,padding='longest',max_length=60, return_tensors="pt").to(torch_device)
    translated = model.generate(**batch,max_length=60,num_beams=num_beams, num_return_sequences=num_return_sequences, temperature=1.5)
    tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
    return tgt_text

create sample outputs for the entire dataset

In [ ]:
%%time
from tqdm import tqdm
num_beams = 1
num_return_sequences = 1
batchsize = 50#900
dataset_name = 'para-1-1'
column_name= 'para-1-1'

for splits in dataset.keys(): #['train', 'val', 'test']
    tgt_text= []
    for x in tqdm(range(0, len(dataset[splits]['text']), batchsize)):
        subset=dataset[splits]['text'][x:x+batchsize]
        batch = tokenizer(subset,truncation=True,padding='max_length',max_length=60, return_tensors="pt").to(torch_device)
        translated = model.generate(**batch,max_length=60,num_beams=num_beams, num_return_sequences=num_return_sequences, temperature=1.5)
        tgt_text.extend(tokenizer.batch_decode(translated, skip_special_tokens=True))

    dataset[splits]=dataset[splits].add_column(column_name, tgt_text)

    dataset.save_to_disk(f'{path}datasets/style/{dataset_name}')
dataset

In [ ]:
from tqdm import tqdm
tgt_text= []
batch_size=2
num_beams = 1 #4 als goldene Mitte?
num_return_sequences = 1 #4 als goldene Mitte?

for x in tqdm(range(0, len(dataset['test']['text']), batch_size)):
    subset=dataset['test']['text'][x:x+batch_size]
    batch = tokenizer(subset,truncation=True,padding='max_length',max_length=60, return_tensors="pt").to(torch_device)
    translated = model.generate(**batch,max_length=60,num_beams=num_beams, num_return_sequences=num_return_sequences, temperature=1.5)
    tgt_text.extend([tokenizer.batch_decode(translated, skip_special_tokens=True)])

dataset['test']=dataset['test'].add_column(f'{model_name}-finetuned-{dataset_name}-lr-{learning_rate}-wd-{weight_decay}', tgt_text)

In [ ]:
dataset['test']['text'][:5]

In [ ]:
tgt_text[:5]

## add [WER](https://huggingface.co/spaces/evaluate-metric/wer) distanz for buckets

In [ ]:
dataset = load_from_disk(f'{path}datasets/style/para-1-1')
dataset

In [ ]:
from tqdm import tqdm
from evaluate import load
wer = load("wer")
column_name = "wer-score"
    
tgt_text= []
for x in tqdm(range(0, len(dataset['test']['text']))):
    #for a single element: predictions and reference are not interchangable
    predictions= [dataset['test']['para-1-1'][x]]
    references = [dataset['test']['text'][x]]
    tgt_text.append(wer.compute(predictions=predictions, references=references))
                    
dataset['test']=dataset['test'].add_column(column_name, tgt_text)

In [ ]:
dataset

In [ ]:
tgt_text[:10] # nicht lemmatized

In [ ]:
save_dataset(dataset,'para-1-1')

## IDM (Insert Delete Modify)

In [ ]:
import random

from transformers import pipeline
from transformers.pipelines.pt_utils import KeyDataset
import datasets
from datasets import Dataset
import numpy as np

unmasker = pipeline('fill-mask', model='bert-base-uncased',device=0)#.to('cuda:0')

In [ ]:
def modify_replace(sentences,replace=1,add_mask=True):#example: sentences=['String sentece two.','I am a sentence.']
    #create batch of sentence to allow batched unmasking if required 
    new_sentences=[]
    for sentence in sentences:
        #unmask only accepts one MASK token, this is a problem when the original sentence already contains one 
        #handle [MASK] like any other word
        sentence = sentence.replace('[MASK]','MASK')
            
        split = sentence.split(' ')
        
        selected_word = random.randrange(0, len(split))

        #find words larger than three characters as long as such words exits and only if we want to replace/delete a word
        while (len(split[selected_word]) < 4) and (replace > 0) and (max([len(x) for x in split])>3):
            selected_word = random.randrange(0, len(split))
            
        split1 = split[:selected_word]
        split2 = split[selected_word+replace:]

        if add_mask:
            sentence = ' '.join(split1 +['[MASK]']+ split2) # unmasker fehlt
        else:
            sentence = ' '.join(split1 + split2)

        new_sentences.append(sentence)
          
    if add_mask:
        #join here -> later only unmask or return
        my_dict = {"text": new_sentences}
        dataset_sentences = Dataset.from_dict(my_dict)
        
        out_sentences=[]
        unmasked_sentences = unmasker(KeyDataset(dataset_sentences, "text"), batch_size=1000,top_k=3,)
        for idx, sentence in enumerate(unmasked_sentences):
            #ensure that the sentence changed
            if (sentences[idx]!=sentence[0]['sequence']) and ('\'' not in sentence[0]['sequence']):
                out_sentences.extend([sentence[0]['sequence']])
            elif (sentences[idx]!=sentence[1]['sequence']) and ('\'' not in sentence[1]['sequence']):
                out_sentences.extend([sentence[1]['sequence']])
            else:
                out_sentences.extend([sentence[2]['sequence']])
        return out_sentences
    else:
        return [out for out in new_sentences]

modify_replace(np.array(["We stated, justified, and",'String sentece two.','I am a sentence.','Ich',]))

In [ ]:
#Test to check if 
def test_function(example_sentence,modify_function,changes_to_length=0,repetition=100):
    for i in range(0,repetition):#debug because size changes created an error
        res = modify_function(np.array(example_sentence))
        if len(res[0].split(' ')) != len(example_sentence[0].split(' '))+changes_to_length:
            print('Not intended modification: ')
            print(example_sentence,len(example_sentence[0].split(' ')))
            print(res,len(res[0].split(' ')))
            print()

example= ['We stated, justified, and']
example_mask= ['We stated, justified, [MASK] [MASK] and']

test_function(example,modify_replace)
test_function(example,modify_replace)

In [ ]:
def modify_delete(sentence):
    return modify_replace(sentence,add_mask=False)

test_function(example,modify_delete,-1)
test_function(example_mask,modify_delete,-1)

In [ ]:
def modify_insert(sentence):
    return modify_replace(sentence,replace = 0)

#results in one additional 'word' if there is a trailing space ' ' doesn't change actual sentences but messes with the test
test_function(example,modify_insert,1)

#also check multiple mask tokens in sentence
test_function(example_mask,modify_insert,1)

In [ ]:
import numpy as np
import math
def modify(sentences):#=['String sentece two.','I am a sentence.']    
    sentences = np.array(sentences)

    max_changes_all = math.ceil(len(sentences[0].split(' '))/2)
    buckets = [(random.randrange(0,6)/10 ,len(x.split(' '))) for x in sentences]
    changes_bucket = [math.floor(length/2)*changes for changes, length in buckets]    
        
    for changes in range(0,max_changes_all):
        #change all senteces that have not the required amount of changes for their bucket
        selected_sentences = [bucket > changes for bucket in changes_bucket]
        #print(selected_sentences)#shows True/False of sentence selection
        r = random.randrange(0,3)
        if r==0:
            sentences[selected_sentences] =  modify_replace(sentences[selected_sentences])
        elif r==1:
            sentences[selected_sentences] =  modify_delete(sentences[selected_sentences])
        else:
            sentences[selected_sentences] =  modify_insert(sentences[selected_sentences])
    return sentences, buckets

modify(np.array(["We stated, justified, and We stated, justified, and",'String sentece two on one.','I am a single sentence.']))

In [ ]:
from datasets import load_from_disk
from datasets import Dataset
dataset= load_from_disk(f'{path}datasets/style/idm')
dataset

In [ ]:
#sort sentences by length for optimization 
#perform the same maximum of changes on sentences with the same length
for splits in dataset.keys(): #['train', 'val', 'test']
    dataset[splits] = dataset[splits].add_column('length',[len(sentence.split(' ')) for sentence in dataset[splits]['text']])
    dataset[splits] = dataset[splits].sort('length',reverse=True)

In [ ]:
dataset['test']['text'][-30000:-29997]

In [ ]:
modify(dataset['test']['text'][-30000:-29997])

In [ ]:
from tqdm import tqdm

column_name = 'idmBucket'
column_name_bucket= 'bucket'
dataset_name = 'idm'
batchsize = 300 #1000~27GB could be improved as this is only required for the longest sentences

for splits in ['test']:#dataset.keys(): #['train', 'val', 'test']
    tgt_text= []
    tgt_bucket = []
    #create batches of sentences -> workaround as split is not defined on dataset.map
    for x in tqdm(range(0, len(dataset[splits]['text']), batchsize)):
        subset=dataset[splits]['text'][x:x+batchsize]
        text, bucket = modify(subset)
        tgt_text.extend(text)
        tgt_bucket.extend([x[0] for x in bucket])
    
    dataset[splits]=dataset[splits].add_column(column_name, tgt_text)
    dataset[splits]=dataset[splits].add_column(column_name_bucket, tgt_bucket)

    save_dataset(dataset,dataset_name)#risky operation, as the old dataset is overwritten
    #dataset.save_to_disk(f'{path}datasets/style/{dataset_name}')

In [ ]:
dataset

In [ ]:
dataset['test']['bucket'][:3]

In [ ]:
dataset['test']['idmBucket'][:3]